In [0]:
#!python -m pip install pyradiomics
import pandas as pd
import numpy as np
import radiomics
from radiomics import featureextractor
  
def fuseCoeff(cooef1, cooef2,cooef3, cooef4, method):

    if (method == 'mean'):
        cooef = (cooef1 + cooef2+ cooef3+ cooef4) / 4
    elif (method == 'min'):
        cooef = np.minimum(cooef1,cooef2)
        cooef = np.minimum(cooef,cooef3)
        cooef = np.minimum(cooef,cooef4)
    elif (method == 'max'):
        cooef = np.maximum(cooef1, cooef2)
        cooef = np.maximum(cooef, cooef3)
        cooef = np.maximum(cooef, cooef4)
    else:
        cooef = []

    return cooef

  extractor = radiomics.featureextractor.RadiomicsFeatureExtractor()




# Params
FUSION_METHOD = 'mean' # Can be 'min' || 'max || anything you choose according theory

rp=
mri=os.listdir(rp)
mri.sort()
  
seg = sitk.ReadImage(rp+"/"+mri[1])
seg = sitk.GetArrayFromImage(seg)
width = 120
height = 120
segt1 = np.zeros((len(seg), width, height))
for idx in range(len(seg)):
  img = seg[idx, :, :]
  img_sm = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)
  segt[idx, :, :] = img_sm

segt1=sitk.GetImageFromArray(segt1)
  
ma_arr = sitk.GetArrayFromImage(segt1)
for l in (1,4):
    ma_arr[ma_arr == l] = 1

segt2= sitk.GetImageFromArray(ma_arr)
segt2.CopyInformation(segt1)


ma_arr = sitk.GetArrayFromImage(segt1)
for l in (1, 4, 2):
    ma_arr[ma_arr == l] = 1

segt3 = sitk.GetImageFromArray(ma_arr)
segt3.CopyInformation(segt1)



inp1= sitk.ReadImage("")
inp2=  sitk.ReadImage("")
inp3 =  sitk.ReadImage("")
inp4 =  sitk.ReadImage("")


I1 = sitk.GetArrayFromImage(inp1)
I2 = sitk.GetArrayFromImage(inp2)
I3 = sitk.GetArrayFromImage(inp3)
I4 = sitk.GetArrayFromImage(inp4)


## Fusion algo

# First: Do wavelet transform on each image
wavelet = 'db1'
cooef1 = pywt.wavedec2(I1[:,:], wavelet)
cooef2 = pywt.wavedec2(I2[:,:], wavelet)
cooef3 = pywt.wavedec2(I3[:,:], wavelet)
cooef4 = pywt.wavedec2(I4[:,:], wavelet)

# Second: for each level in both image do the fusion according to the desire option
fusedCooef = []

for i in range(len(cooef1)-1):

    # The first values in each decomposition is the apprximation values of the top level
    if(i == 0):

        fusedCooef.append(fuseCoeff(cooef1[0],cooef2[0], cooef3[0], cooef4[0], FUSION_METHOD))

    else:

        # For the rest of the levels we have tupels with 3 coeeficents
        c1 = fuseCoeff(cooef1[i][0], cooef2[i][0], cooef3[i][0], cooef4[i][0], FUSION_METHOD)
        c2 = fuseCoeff(cooef1[i][1], cooef2[i][1], cooef3[i][1], cooef4[i][1], FUSION_METHOD)
        c3 = fuseCoeff(cooef1[i][2], cooef2[i][2], cooef3[i][2], cooef4[i][2], FUSION_METHOD)

        fusedCooef.append((c1,c2,c3))

# Third: After we fused the cooefficent we nned to transfor back to get the image
fusedImage = pywt.waverec2(fusedCooef, wavelet)

# Forth: normmalize values to be in uint8
fusedImage = np.multiply(np.divide(fusedImage - np.min(fusedImage),(np.max(fusedImage) - np.min(fusedImage))),255)
fusedImage = fusedImage.astype(np.uint8)

#fusedImage=sitk.ReadImage(fusedImage)


fusedImage=sitk.GetImageFromArray(fusedImage)

result = extractor.execute(fusedImage, segt1, label=1)
data= pd.DataFrame(columns=result.keys())
data= data.iloc[:, 22:]
for featureName in data.columns:
    data.loc[0, featureName]=result[featureName]

result = extractor.execute(fusedImage, segt2, label=1)
data1= pd.DataFrame(columns=result.keys())
data1= data1.iloc[:, 22:]
for featureName in data1.columns:
    data1.loc[0, featureName]=result[featureName]

result = extractor.execute(fusedImage, segt3, label=1)
data2= pd.DataFrame(columns=result.keys())
data2= data2.iloc[:, 22:]
for featureName in data2.columns:
    data2.loc[0, featureName]=result[featureName]

data1.columns = [str(col) + ' ROI 1' for col in data1.columns]
data2.columns = [str(col) + ' ROI 2' for col in data2.columns]
data.columns = [str(col) + ' ROI 3' for col in data.columns]
  
data1["GRADE"]=
data2["GRADE"]=
data["GRADE"]=
  

datac=data1.merge(data2, how="inner", on=["GRADE"])
datac=datac.merge(data, how="inner", on=["GRADE"])

fused_data=datac
fused_data